In [3]:
import pandas as pd
import numpy as np
import random
import json
from collections import Counter
import warnings
import random
warnings.filterwarnings('ignore')

df = pd.read_csv('data/data_compiled_final_may_2023.csv')
df['uploader']=[i.split(' ')[0] for i in df['uploader']]
df = df[['ids','upload_date','duration','comment_count','view_count','uploader']]
df = df.fillna(0)
df['upload_date'] = [str(int(i)) for i in df['upload_date']]
df['upload_date'] = pd.to_datetime(df['upload_date'],format='%Y%m%d')
df_sentiment = pd.read_csv('data/data_compiled_final_may_2023_sentiment.csv')
senti_dict = dict(zip(df_sentiment.ids, df_sentiment.sentiment))
df['sentiment'] = [senti_dict[i] for i in df['ids'].values]


In [9]:
'''
uploader
'''
df_uploader = df[['uploader','upload_date','duration','comment_count','view_count','sentiment']]
df_uploader.to_csv('/Users/kowshik/projects/personal/alphapolitica_streamlit/UIdata/uploader.csv',index= False)


'''
dummy data
'''

df_original = pd.read_csv('data/intermediate_results')
df_original['translated'] = df_original['translated'].fillna('')
tags = [i for i in df_original.columns[10:] if 'proba_' not in i]
df_original['english_translated'] = [1 if 'a' in i else 0 for i in df_original['translated'].values]

df_tags = df.copy()

for tag in tags:
    found = []
    for i in range(len(df_original)):
        proba = df_original['proba_'+tag].values[i]
        regex = df_original[tag].values[i]
        eng = df_original['english_translated'].values[i]
        if regex == 1:
            found.append(1)
        else:
            if eng == 0:
                found.append(0)
            elif proba >0.7:
                found.append(1)
            else:
                found.append(0)
    df_tags[tag] = found

''' 
Faces
'''

with open("data/facerec_results.txt", "r") as fp:
    person_dict = json.load(fp)
df_attributes = pd.read_excel('data/alphapolitica_faces_attributes.xlsx')

with open("data/facerec_results.txt", "r") as fp:
    person_dict = json.load(fp)
faces_id = list(person_dict.keys())
person_dict_updated = {}
for id_ in faces_id:
    output = person_dict[id_]
    df_id = pd.DataFrame()
    df_id['output'] = output
    df_id['output_id'] = output = [i[:11] for i in output]
    df_id['rank'] = np.arange(len(df_id))
    df_id['count'] = 1
    df_grouped = df_id.groupby('output_id').aggregate({'rank':np.min, 'count':np.sum}).reset_index()
    df_grouped = df_grouped.sort_values('rank')
    df_ranked = df_grouped[df_grouped['rank'] < len(df_id)/1.5]
    df_ranked = df_ranked[df_ranked['count'] > 1]
    person_dict_updated[id_] = df_ranked['output_id'].values
df_attributes = pd.read_excel('data/alphapolitica_faces_attributes.xlsx')
df_attributes = df_attributes[df_attributes.image_id.isin(list(person_dict_updated.keys()))]
df_attributes['number'] = [len(person_dict_updated[i]) for i in df_attributes.image_id]
df_ = df_attributes[df_attributes['Gender'] == 'F']
to_drop = df_[df_.number>800]['image_id'].values
to_drop = list(set(to_drop)- set(['nagari.png']))


df_faces = df.copy()
faces_id = set(list(person_dict_updated.keys()))-set(to_drop)
lis_ = []
for f_id in faces_id:
    if f_id in df_attributes.image_id.values:
        faces_dict =  dict(Counter([i[:11] for i in person_dict_updated[f_id]]))
        df_faces[f_id] = [faces_dict[i] if i in faces_dict.keys() else 0 for i in df_faces.ids]

df_attributes = pd.read_excel('data/alphapolitica_faces_attributes.xlsx')
df_attributes['Age'] = [int(i) for i in df_attributes['Age'].fillna(np.mean(df_attributes['Age']))]
coords = []
df_attributes['loc'] = df_attributes['loc'].fillna('')
for i in range(len(df_attributes)):
    loc = df_attributes['loc'].values[i].replace(' ','')
    if loc != '':
        if '°N' in loc:
            lat = loc.split('°N')[0].strip()
            long = loc.split('°N')[1].split('°E')[0].strip()
            coords.append(lat+long)
        else:
            coords.append(loc)
    else:
        coords.append('0,0')

df_attributes['coords'] = coords
del df_attributes['loc']
df_attributes.to_excel('/Users/kowshik/projects/personal/alphapolitica_streamlit/UIdata/alphapolitica_faces_attributes_coords.xlsx', index = False)
df_faces.to_csv('/Users/kowshik/projects/personal/alphapolitica_streamlit/UIdata/faces.csv', index = False)
df_tags.to_csv('/Users/kowshik/projects/personal/alphapolitica_streamlit/UIdata/tags.csv', index = False)


In [5]:
import json
with open("data/facerec_results.txt", "r") as fp:
    person_dict = json.load(fp)

In [6]:
list(person_dict.keys())

['palakonda.png',
 'tirupati_l2.png',
 'vijayawada_central.png',
 'mada.png',
 'kavali.png',
 'mantralayam.png',
 'puttaparthi.png',
 'pathapatnam.png',
 'cheepurupali.png',
 'tenali.png',
 'panyam.png',
 'guntur_east.png',
 'rajahmundry_rural.png',
 'tiruvuru.png',
 'machillipatnam.png',
 'pileru.png',
 'pattikonda.png',
 'gannavaram.png',
 'yerragondapalem.png',
 'gajapathinagram.png',
 'narsapuram.png',
 'srisailam.png',
 'paturi_nagabhusnam.png',
 'chodavaram.png',
 'venkatagiri.png',
 'madugula.png',
 'denduluru.png',
 'vizianagaram_.png',
 'Etcherla.png',
 'araku_valley.png',
 'amalapuram.png',
 'achanta.png',
 'badvel.png',
 'amalapuram_l.png',
 'narsapuram_l.png',
 'prathipadu_.png',
 'kurnool_l.png',
 'vishakapatnam_east.png',
 'vijay_kumar.png',
 'undi.png',
 'visakapatnam.png',
 'tadikonda.png',
 'nellore_rural.png',
 'vinukonda.png',
 'vishakapatnam_west.png',
 'mandanapalli.png',
 'kandukur.png',
 'guntur_west.png',
 'penamaluru.png',
 'pamarru.png',
 'kakinada_.png',
 'na

In [3]:
import pandas as pd
import numpy as np
import random
import json
from collections import Counter
import warnings
import random
warnings.filterwarnings('ignore')
df = pd.read_csv('data/data_compiled_final_may_2023.csv')
df['uploader']=[i.split(' ')[0] for i in df['uploader']]
df = df[['ids','upload_date','duration','comment_count','view_count','uploader']]
df = df.fillna(0)
df['upload_date'] = [str(int(i)) for i in df['upload_date']]
df['upload_date'] = pd.to_datetime(df['upload_date'],format='%Y%m%d')
df_sentiment = pd.read_csv('data/data_compiled_final_may_2023_sentiment.csv')
senti_dict = dict(zip(df_sentiment.ids, df_sentiment.sentiment))
df['sentiment'] = [senti_dict[i] for i in df['ids'].values]

with open("data/facerec_results.txt", "r") as fp:
    person_dict = json.load(fp)
df_attributes = pd.read_excel('data/alphapolitica_faces_attributes.xlsx')

df_faces = df.copy()
faces_id = list(person_dict.keys())
lis_ = []
for f_id in faces_id:
    if f_id in df_attributes.image_id.values:
        faces_dict =  dict(Counter([i[:11] for i in person_dict[f_id]]))
        df_faces[f_id] = [faces_dict[i] if i in faces_dict.keys() else 0 for i in df_faces.ids]

In [20]:
import json
import pandas as pd
import numpy as np
with open("data/facerec_results.txt", "r") as fp:
    person_dict = json.load(fp)
faces_id = list(person_dict.keys())
person_dict_updated = {}
for id_ in faces_id:
    output = person_dict[id_]
    df = pd.DataFrame()
    df['output'] = output
    df['output_id'] = output = [i[:11] for i in output]
    df['rank'] = np.arange(len(df))
    df['count'] = 1
    df_grouped = df.groupby('output_id').aggregate({'rank':np.min, 'count':np.sum}).reset_index()
    df_grouped = df_grouped.sort_values('rank')
    df_ranked = df_grouped[df_grouped['rank'] < len(df)/2]
    df_ranked = df_ranked[df_ranked['count'] > 2]
    person_dict_updated[id_] = df_ranked['output_id'].values
df_attributes = pd.read_excel('data/alphapolitica_faces_attributes.xlsx')
df_attributes = df_attributes[df_attributes.image_id.isin(list(person_dict_updated.keys()))]
df_attributes['number'] = [len(person_dict_updated[i]) for i in df_attributes.image_id]
df_ = df_attributes[df_attributes['Gender'] == 'F']
to_drop = df_[df_.number>800]['image_id'].values

array(['rajmundry_city.png', 'chilakuluripet.png', 'pattikonda.png',
       'anakapalli.png', 'amalapuram_l.png'], dtype=object)

In [14]:
df_attributes = pd.read_excel('data/alphapolitica_faces_attributes.xlsx')
df_attributes['Age'] = [int(i) for i in df_attributes['Age'].fillna(np.mean(df_attributes['Age']))]
df_attributes['constituency'] = df_attributes['Name']
#Add real coordinates and constituency here


poly = Polygon([(17.298641, 79.687922),
 (16.861628, 81.643629),
 (14.884918, 79.717799),
 (14.994391, 77.257505)])

def polygon_random_points (poly, num_points):
    min_x, min_y, max_x, max_y = poly.bounds
    points = []
    while len(points) < num_points:
            random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
            if (random_point.within(poly)):
                points.append(random_point)
    return points

points = polygon_random_points(poly,len(df_attributes))
coords = []
for p in points:
    coords.append(str(p.x) +","+str(p.y))
df_attributes['coords'] = coords
df_attributes.to_excel('/Users/kowshik/projects/personal/alphapolitica_streamlit/UIdata/alphapolitica_faces_attributes_coords.xlsx', index = False)


In [3]:
import pandas as pd
df = pd.read_csv('data/intermediate_results')